In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./EEG_data.csv')
data = pd.read_csv('./demographic_info.csv')

In [3]:
data = data.rename(columns={'subject ID': 'SubjectID',
                   ' gender': 'gender', ' age': 'age', ' ethnicity': 'ethnicity'})
df = df.merge(data, how='inner', on='SubjectID')
# df.head()

In [4]:
df['gender'] = df['gender'].replace({'M': 1, 'F': 0})
df['ethnicity'] = df['ethnicity'].replace(
    {'Han Chinese': 0, 'Bengali': 1, 'English': 2})
df.drop(columns=['SubjectID', 'VideoID', 'predefinedlabel'], inplace=True)

In [5]:
df.columns

Index(['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'user-definedlabeln', 'age',
       'ethnicity', 'gender'],
      dtype='object')

In [6]:
top_fea = df.drop(['user-definedlabeln'], axis=1)

In [7]:
top_fea.columns

Index(['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'age', 'ethnicity', 'gender'],
      dtype='object')

In [8]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [9]:
from sklearn.model_selection import train_test_split
y = df['user-definedlabeln']
X = df.drop(columns=['user-definedlabeln'])
X_train, x_test, Y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.27)

In [10]:
scaler = StandardScaler()
svm_classifier = SVC()

pipeline = make_pipeline(scaler, svm_classifier)

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [11]:
y_test.head()

772      1.0
10196    0.0
385      1.0
4269     0.0
8        0.0
Name: user-definedlabeln, dtype: float64

In [14]:

received_array = np.array([41.0,78.0,195.0,2508980.0,474498.0,344011.0,154192.0,52440.0,98050.0,241122.0,31639.0,25,0,1])
# Convert the array into a DataFrame with the same structure
columns_to_select = ['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'age', 'ethnicity', 'gender']
# Create a new DataFrame with the selected columns
received_dataframe = pd.DataFrame(data=[received_array], columns=columns_to_select)
# received_dataframe
X_new = received_dataframe
X_new_standardized = scaler.transform(X_new)
predictions = svm_classifier.predict(X_new_standardized)
predictions


array([1.])

In [16]:
import joblib

joblib.dump(pipeline, 'svm_model.joblib')

['svm_model.joblib']

## Testing server


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


app = Flask(__name__)
CORS(app)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    return jsonify({'data': data})


if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


## Final response


In [18]:
from flask import Flask, request, jsonify
import numpy as np
import joblib
from flask_cors import CORS

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_model = loaded_pipeline.named_steps['svc']

import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

app = Flask(__name__)
CORS(app)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    x_test = data['features']
    received_array = np.array(x_test)
    columns_to_select = ['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'age', 'ethnicity', 'gender']
    received_dataframe = pd.DataFrame(data=[received_array], columns=columns_to_select)
    X_new = received_dataframe
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)[0]

    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    confidence_percentage = round(float(sigmoid(confidence_scores) * 100),2)

    return jsonify({'prediction': prediction.tolist(), 'confidence': confidence_percentage})


if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Feb/2024 22:11:42] "POST /predict HTTP/1.1" 200 -
